In [5]:
# Import necessary libraries

import pandas as pd

In [6]:
mothers_hmis_df = pd.read_csv('data/mothers_hmis.csv')
facilities_lookup_df = pd.read_csv('data/facilities_lookup.csv')
prompts_engagement_df = pd.read_csv('data/prompts_engagement.csv')

In [7]:
mothers_hmis_df.head()

,mother_id,facility_id,county,residence_type,age,age_group,parity,delivery_date,delivery_mode,complication_delivery,postpartum_complication_6w,postnatal_visit_date_1,postnatal_visit_6w,prompts_enrolled,prompts_mother_id
0,1,F028,Kilifi,rural,36,35-49,4,2024-10-17,CS,none,0,2024-11-18,1,0,NaN
1,2,F007,Kiambu,peri_urban,34,25-34,4,2024-07-23,SVD,other,0,2024-08-30,1,1,P00001
2,3,F002,Nairobi,urban,30,25-34,1,2024-03-21,SVD,sepsis,0,2024-04-25,1,0,NaN
3,4,F019,Kajiado,peri_urban,17,15-19,0,2024-03-23,SVD,none,0,NaN,0,1,P00002
4,5,F019,Kajiado,peri_urban,24,20-24,2,2024-11-06,SVD,none,0,2024-11-23,1,1,P00003


##### 1. Define KPIs

In [31]:
# 1. Overall PNC 6‑week coverage (at least one postnatal visit within 6 weeks of delivery)

total_mothers = len(mothers_hmis_df)
total_mothers_with_pnc_6w = mothers_hmis_df["postnatal_visit_6w"].sum()

pnc_6_week_coverage = 100 * total_mothers_with_pnc_6w / total_mothers

print(f"The total coverage of PNC 6-week visits is {pnc_6_week_coverage:.2f}%")

The total coverage of PNC 6-week visits is 57.96%


In [30]:
# 2. PNC 6‑week coverage by age group (only 1 = attended)

pnc_by_age = (
    mothers_hmis_df[mothers_hmis_df["postnatal_visit_6w"] == 1]
    .groupby("age_group")["postnatal_visit_6w"]
    .count()
)
print(pnc_by_age)

age_group
15-19     303
20-24     813
25-34    1192
35-49     590
Name: postnatal_visit_6w, dtype: int64


In [32]:
# 3. PNC 6-week coverage by residence type

pnc_by_residence = (
    mothers_hmis_df[mothers_hmis_df["postnatal_visit_6w"] == 1]
    .groupby("residence_type")["postnatal_visit_6w"]
    .count()
)
print(pnc_by_residence)

residence_type
peri_urban    1314
rural         1180
urban          404
Name: postnatal_visit_6w, dtype: int64


In [ ]:
# 4. PNC 6-week coverage by parity

pnc_by_county = (
    mothers_hmis_df[mothers_hmis_df["postnatal_visit_6w"] == 1]
    .groupby("county")["postnatal_visit_6w"]
    .count()
)
pnc_by_county

county
Kajiado     661
Kiambu      312
Kilifi      564
Kisii       261
Machakos    341
Makueni     355
Nairobi     404
Name: postnatal_visit_6w, dtype: int64

In [ ]:
# 5. PNC 6-week coverage by parity

pnc_by_parity = (
    mothers_hmis_df[mothers_hmis_df["postnatal_visit_6w"] == 1]
    .groupby("parity")["postnatal_visit_6w"]
    .count()
)
pnc_by_parity

parity
0    744
1    734
2    572
3    424
4    298
5    126
Name: postnatal_visit_6w, dtype: int64